In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import os
import nibabel as nib
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from utils import process_file
from CNN1DModel import CNN1DModel

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f'Using device: {device}')


Using device: mps


In [28]:

# Data loaders

REMOVE_TIME_POINT = 5  # Remove the first samples from the fMRI data
DATA_THRESHOLD = 1
ZERO_COUNT_THRESHOLD = 43 # 10%x430

class TimeSeriesDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
        self.targets = torch.tensor(targets, dtype=torch.float32)
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]


def load_dataset_parallel(input_dir, target_dir, num_workers=1):
    # List all files in the directories, sorted for consistency
    input_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(('.nii', '.nii.gz'))])
    target_files = sorted([os.path.join(target_dir, f) for f in os.listdir(target_dir) if f.endswith(('.nii', '.nii.gz'))])

    inputs = []
    targets = []

    # Use parallel processing to load inputs and targets
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        inputs = list(executor.map(
            process_file, 
            input_files, 
            [False] * len(input_files), 
            [REMOVE_TIME_POINT] * len(input_files), 
            [DATA_THRESHOLD] * len(input_files), 
            [ZERO_COUNT_THRESHOLD] * len(input_files)
        ))
        targets = list(executor.map(process_file, target_files, [True] * len(target_files)))

    # Stack the loaded data
    inputs = np.vstack([x for x in inputs if x.size > 0]).astype(np.float32)
    targets = np.vstack([x for x in targets if x.size > 0]).astype(np.float32)

    # Save original length
    original_length = inputs.shape[0]

    # Create a boolean mask for rows where all values are zero in inputs or targets
    non_zero_mask = ~((inputs == 0).all(axis=1) | (targets == 0).all(axis=1))

    # Apply the mask to filter out rows
    inputs = inputs[non_zero_mask, :]  # Keep rows in inputs
    targets = targets[non_zero_mask, :]  # Keep rows in targets

    # Calculate the number of removed samples
    removed_samples = original_length - inputs.shape[0]

    print(f"Final input shape: {inputs.shape}")
    print(f"Final target shape: {targets.shape}")
    print(f"Samples included: {inputs.shape[0]}")
    print(f"Samples removed: {removed_samples}")

    return inputs, targets


In [29]:
# Load data 

def load_cached_data(input_file, target_file, input_dir, target_dir):
    if os.path.exists(input_file) and os.path.exists(target_file):
        inputs = np.load(input_file)
        targets = np.load(target_file)
    else:
        inputs, targets = load_dataset_parallel(input_dir, target_dir)
        np.save(input_file, inputs)  # Save inputs
        np.save(target_file, targets)  # Save targets
    
    return inputs, targets

BATCH_SIZE = 32

# Common directory
BASE_DIR = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project"

# Subdirectories
TRAIN_INPUT_DIR = os.path.join(BASE_DIR, "func/registered/main_data/training")
VAL_INPUT_DIR = os.path.join(BASE_DIR, "func/registered/main_data/validation")
TEST_INPUT_DIR = os.path.join(BASE_DIR, "func/registered/main_data/testing")

TRAIN_TARGET_DIR = os.path.join(BASE_DIR, "CVR_MAPS/registered/training")
VAL_TARGET_DIR = os.path.join(BASE_DIR, "CVR_MAPS/registered/validation")
TEST_TARGET_DIR = os.path.join(BASE_DIR, "CVR_MAPS/registered/testing")

# Load and preprocess the data
print("Loading training data...")
train_inputs, train_targets = load_cached_data("train_inputs.npy", "train_targets.npy", TRAIN_INPUT_DIR, TRAIN_TARGET_DIR)

print("Loading validation data...")
val_inputs, val_targets = load_cached_data("val_inputs.npy", "val_targets.npy", VAL_INPUT_DIR, VAL_TARGET_DIR)

print("Loading test data...")
test_inputs, test_targets = load_cached_data("test_inputs.npy", "test_targets.npy", TEST_INPUT_DIR, TEST_TARGET_DIR)

# Create datasets and dataloaders
train_dataset = TimeSeriesDataset(train_inputs, train_targets)
val_dataset = TimeSeriesDataset(val_inputs, val_targets)
test_dataset = TimeSeriesDataset(test_inputs, test_targets)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


Loading training data...
Final input shape: (2187083, 430)
Final target shape: (2187083, 1)
Samples included: 2187083
Samples removed: 4841653
Loading validation data...
Final input shape: (353280, 430)
Final target shape: (353280, 1)
Samples included: 353280
Samples removed: 711680
Loading test data...
Final input shape: (621633, 430)
Final target shape: (621633, 1)
Samples included: 621633
Samples removed: 1401791


In [30]:

# Build model
INPUT_SIZE = 435 - REMOVE_TIME_POINT # Number of time points
LEARNING_RATE = 1e-3
model = CNN1DModel(input_size=INPUT_SIZE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


TypeError: 'module' object is not callable. Did you mean: 'CNN1DModel.CNN1DModel(...)'?

In [ ]:
# Parameters
EPOCHS = 20
MODEL_SAVE_PATH = "best_model.pth"
PATIENCE = 5  # Stop training if validation loss doesn't improve for 5 consecutive epochs

def train_model(model, train_loader, val_loader, criterion, optimizer, device):
    model.to(device)
    best_val_loss = float("inf")  # Initialize with a large value
    epochs_without_improvement = 0  # Counter for epochs without improvement

    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}")
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
        
        val_loss /= len(val_loader.dataset)
        print(f"Validation Loss: {val_loss:.4f}")

        # Check for improvement
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0  # Reset counter
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"Model saved with Validation Loss: {val_loss:.4f}")
        else:
            epochs_without_improvement += 1
            print(f"No improvement for {epochs_without_improvement} epoch(s).")

        # Early stopping
        if epochs_without_improvement >= PATIENCE:
            print(f"Early stopping triggered. No improvement for {PATIENCE} consecutive epochs.")
            break


In [ ]:
# Train the model
print("Training the model...")
train_model(model, train_loader, val_loader, criterion, optimizer, device)


In [ ]:
# Evaluate on test data
print("Evaluating the model on test data...")
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)

test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")